# Laboratorio de regresión - 4

|                |                      |
:----------------|----------------------|
| **Nombre**     | Gonzalo Cano Padilla |
| **Fecha**      | 9 de septiembre 2025 |
| **Expediente** | 745901               |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

In [1]:
import pandas as pd
from sklearn.linear_model import Lasso, Ridge
import numpy as np

datos = pd.read_csv('Advertising.csv',index_col=0)
datos

,TV,radio,newspaper,sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9
...,...,...,...,...
196,38.2,3.7,13.8,7.6
197,94.2,4.9,8.1,9.7
198,177.0,9.3,6.4,12.8
199,283.6,42.0,66.2,25.5


## Regresión sin penalización

In [2]:
datos.describe()

,TV,radio,newspaper,sales
count,200.000000,200.000000,200.000000,200.000000
mean,147.042500,23.264000,30.554000,14.022500
std,85.854236,14.846809,21.778621,5.217457
min,0.700000,0.000000,0.300000,1.600000
25%,74.375000,9.975000,12.750000,10.375000
50%,149.750000,22.900000,25.750000,12.900000
75%,218.825000,36.525000,45.100000,17.400000
max,296.400000,49.600000,114.000000,27.000000


In [3]:
y = datos['sales'].values
n=len(datos)
unos = np.ones([n,1])
x = datos.drop(columns=['sales'])

In [4]:
# Regresión sin penalización
import statsmodels.api as sm

X=np.hstack([unos,x])
ols = sm.OLS(y,X)
results = ols.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Thu, 11 Sep 2025   Prob (F-statistic):           1.58e-96
Time:                        11:25:39   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9389      0.312      9.422      0.000       2.324       3.554
x1             0.0458      0.001     32.809      0.000       0.043       0.049
x2             0.1885      0.009     21.893      0.000       0.172       0.206
x3            -0.0010      0.006     -0.177      0.860      -0.013       0.011
==============================================================================
Omnibus:                       60.414   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.241
Skew:                          -1.327   Prob(JB):                     1.44e-33
Kurtosis:                       6.332   Cond. No.                         454.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Regresión con penalización Ridge L2

In [6]:
from scipy import stats

y = datos['sales']
x = datos.drop(columns=['sales'])
n,p = x.shape

L2 = Ridge(alpha=0.01, fit_intercept=True)
L2.fit(x,y)

# RSS
y_hat = L2.predict(x)
RSS_L2 = np.sum((y-y_hat)**2)

#RSE
RSE_L2 = np.sqrt(RSS_L2/(n-p))

# Var_beta
X = np.column_stack([np.ones(n),x]) # Es la x con el intercepto
var_beta = np.linalg.inv(X.T @ X) * RSE_L2**2

std_beta = np.sqrt(var_beta.diagonal())

# Estadisticos t
beta = np.r_[L2.intercept_, L2.coef_]   # vector (p+1,)
t_L2 = beta / std_beta

# P-values
p_L2 = 2 * (1 - stats.t.cdf(np.abs(t_L2), n-p-1))

# R^2
R2 = L2.score(x, y)

print("----Resultados Ridge(L2)----")
print(f"Intercepto: {L2.intercept_}")
print(f"Coefficientes: {L2.coef_}")
print(f"RSS: {RSS_L2}")
print(f"RSE: {RSE_L2}")
print(f"Var beta: {var_beta}")
print(f"Std beta: {std_beta}")
print(f"P-Values: {p_L2}")
print(f"R^2: {R2}")

----Resultados Ridge(L2)----
Intercepto: 2.938890150364781
Coefficientes: [ 0.04576465  0.18852997 -0.00103748]
RSS: 556.8252629022802
RSE: 1.6812269856176278
Var beta: [[ 9.67929065e-02 -2.64378468e-04 -1.10982708e-03 -5.88021132e-04]
 [-2.64378468e-04  1.93586026e-06 -4.44770227e-07 -3.24937184e-07]
 [-1.10982708e-03 -4.44770227e-07  7.37769375e-05 -1.77102660e-05]
 [-5.88021132e-04 -3.24937184e-07 -1.77102660e-05  3.42937860e-05]]
Std beta: [0.31111558 0.00139135 0.00858935 0.00585609]
P-Values: [0.         0.         0.         0.85956348]
R^2: 0.897210638178935


## Regresión con penalización Laso L1

In [7]:
L1 = Lasso(alpha=0.01, fit_intercept=True)
L1.fit(x,y)

# RSS
y_hat = L1.predict(x)
RSS_L1 = np.sum((y-y_hat)**2)

#RSE
RSE_L1 = np.sqrt(RSS_L1/(n-p))

# Var_beta
X = np.column_stack([np.ones(n),x]) # Es la x con el intercepto
var_beta = np.linalg.inv(X.T @ X) * RSE_L1**2

std_beta = np.sqrt(var_beta.diagonal())

# Estadisticos t
beta = np.r_[L1.intercept_, L1.coef_]   # vector (p+1,)
t_L1 = beta / std_beta

# P-values
p_L1 = 2 * (1 - stats.t.cdf(np.abs(t_L1), n-p-1))

# R^2
R2 = L1.score(x, y)

print("----Resultados Ridge(L2)----")
print(f"Intercepto: {L1.intercept_}")
print(f"Coefficientes: {L1.coef_}")
print(f"RSS: {RSS_L1}")
print(f"RSE: {RSE_L1}")
print(f"Var beta: {var_beta}")
print(f"Std beta: {std_beta}")
print(f"P-Values: {p_L1}")
print(f"R^2: {R2}")

----Resultados Ridge(L2)----
Intercepto: 2.939434895672264
Coefficientes: [ 0.0457633   0.1884668  -0.00100074]
RSS: 556.8254627216263
RSE: 1.6812272872756784
Var beta: [[ 9.67929413e-02 -2.64378562e-04 -1.10982748e-03 -5.88021343e-04]
 [-2.64378562e-04  1.93586096e-06 -4.44770387e-07 -3.24937300e-07]
 [-1.10982748e-03 -4.44770387e-07  7.37769640e-05 -1.77102724e-05]
 [-5.88021343e-04 -3.24937300e-07 -1.77102724e-05  3.42937983e-05]]
Std beta: [0.31111564 0.00139135 0.00858935 0.00585609]
P-Values: [0.         0.         0.         0.86448697]
R^2: 0.8972106012924924


### Conclusión

Los resultados muestran que los tres modelos (OLS, Ridge y Lasso) alcanzan un \(R^2\) cercano al 0.897, por lo que explican alrededor del 90 % de la variabilidad de las ventas. La inversión en **TV** y **radio** resulta significativa en todos los casos, confirmando su impacto positivo, mientras que **newspaper** no presenta un efecto estadísticamente relevante. Con un alpha bajo, Ridge y Lasso apenas modifican los resultados, aunque en escenarios con mayor penalización podrían ayudar a reducir la complejidad del modelo o eliminar variables irrelevantes.